## What is Design Pattern?
 - In software engineering, a software design pattern is a general, reusable solution to a commonly occurring problem within a given context in software design.
 - Design patterns are generally language-independent (languages supporting object-oriented programming \[OOP\]), and some patterns have been implemented and been a part of Python syntax (iterator).
 - Four categories of design patterns:
     - Creational patterns
     - Structural patterns
     - Behavioral patterns
     - Concurrency patterns
 - Today we introduce four basic common patterns:
     - Decorator (Structural)
     - Adapter (Structural)
     - Observer (Behavioral)
     - Factory Mode (Creational)


## Decorator
 - Decorator pattern is a design pattern that allows behavior to be added to an individual object, dynamically, without affecting the behavior of other objects from the same class.
 - Example: You have already created a bunch of functions, now you want to add a timer to each of them. Brute way is modify each function. A smart way is to use decorator.

In [1]:
import time

def myfun1(n):
    for i in range(n):
        time.sleep(1)
        print(f'Wait: {i}')
    return n+1

myfun1(3)

Wait: 0
Wait: 1
Wait: 2


4

In [2]:
# Brute way:
def myfun1(n):
    start_time = time.time()
    
    for i in range(n):
        time.sleep(1)
        print(f'Wait: {i}')  
        
    print(f'Time consumption {time.time()-start_time}')
    
    return n+1
    
myfun1(3)

Wait: 0
Wait: 1
Wait: 2
Time consumption 3.007308006286621


4

In [3]:
# Another way
import time

def myfun1(n):
    for i in range(n):
        time.sleep(1)
        print(f'Wait: {i}')
    return n+1

def myfun2(n):
    start_time = time.time()
    returns = myfun1(n)
    print(f'Time consumption {time.time()-start_time}')
    return returns

myfun2(3)

Wait: 0
Wait: 1
Wait: 2
Time consumption 3.0123860836029053


4

In [4]:
# A decorator
def myTimeDecorator(func):
    def wrapper(n):
        start_time = time.time()
        returns = func(n)
        print(f'Time consumption {time.time()-start_time}')
        return returns
    return wrapper


def myfun1(n):
    for i in range(n):
        time.sleep(1)
        print(f'Wait: {i}')
    return n+1

myfun1 = myTimeDecorator(myfun1)

myfun1(3)

Wait: 0
Wait: 1
Wait: 2
Time consumption 3.009680986404419


4

In [5]:
# Write decorator in Python with @
def myTimeDecorator(func):
    def wrapper(n):
        start_time = time.time()
        returns = func(n)
        print(f'Time consumption {time.time()-start_time}')
        return returns
    return wrapper

@myTimeDecorator
def myfun1(n):
    for i in range(n):
        time.sleep(1)
        print(f'Wait: {i}')
    return n+1

myfun1(3)

Wait: 0
Wait: 1
Wait: 2
Time consumption 3.0092110633850098


4

In [6]:
# Another example:
# Sometimes, you need to write a inherited class from others' work. 
# To make sure that you override the correct attributes, you can use a decorator to check it.


class A:
    def myfun1(self, n):
        raise NotImplementedError

class A_child(A):
    def myfun1(self, n):
        for i in range(n):
#             time.sleep(1)
            print(f'Wait: {i}')
        return n+1
    
a = A_child()
a.myfun1(3)

Wait: 0
Wait: 1
Wait: 2


4

In [7]:
# Reference: https://stackoverflow.com/questions/1167617/in-python-how-do-i-indicate-im-overriding-a-method/8313042#8313042

def overrides(interface_class):
    def overrider(method):
        assert method.__name__ in dir(interface_class), \
        f'This attribute {method} is not an attribute in class {interface_class}'
        return method
    return overrider

In [21]:
class A_child(A):
    @overrides(A)
    def myfan1(self, n):
        for i in range(n):
#             time.sleep(1)
            print(f'Wait: {i}')
        return n+1

AssertionError: This attribute <function A_child.myfan1 at 0x000001C00D13F9D8> is not an attribute in class <class '__main__.A'>

## Adapter
 - In software engineering, the adapter pattern is a software design pattern (also known as wrapper, an alternative naming shared with the decorator pattern) that allows the interface of an existing class to be used as another interface. It is often used to make existing classes work with others without modifying their source code. (Wikipedia)
 - Assume that now we have different classes which have differnet attributes name, but their funcitonality are similiar, how can we unify those interfaces? (Different databases has different interfaces, but their funcitonality are similar).

In [23]:
# Reference: https://github.com/PJUllrich/Design-Patterns

class Elf:
    def nall_nin(self):
        print('Elf says: Calling the Overlord ...')


class Dwarf:
    def estver_narho(self):
        print('Dwarf says: Calling the Overlord ...')


class Human:
    def ring_mig(self):
        print('Human says: Calling the Overlord ...')

In [24]:
oneMinion = Elf()
oneMinion.nall_nin()

oneMinion = Dwarf()
oneMinion.estver_narho()

oneMinion = Human()
oneMinion.ring_mig()

Elf says: Calling the Overlord ...
Dwarf says: Calling the Overlord ...
Human says: Calling the Overlord ...


In [25]:
# A better approach, but if there are multiple attributes in our target classes, this method would be tedious.
def intepretor(minion):
    if type(minion) == Elf:
        minion.nall_nin()
    elif type(minion) == Dwarf:
        minion.estver_narho()
    elif type(minion) == Human:
        minion.ring_mig()
    else:
        raise TypeError
        
intepretor(Elf())
intepretor(Dwarf())
intepretor(Human())

Elf says: Calling the Overlord ...
Dwarf says: Calling the Overlord ...
Human says: Calling the Overlord ...


In [63]:

class MinionAdapter:
    _initialised = False

    def __init__(self, minion, **adapted_methods):
        self.minion = minion
#         print(self._initialised)
        for key, value in adapted_methods.items():
            func = getattr(self.minion, value)
            self.__setattr__(key, func)

        self._initialised = True

    def __getattr__(self, attr):
        """Attributes not in Adapter are delegated to the minion"""
        return getattr(self.minion, attr)

    def __setattr__(self, key, value):
        """Set attributes normally during initialisation"""
        if not self._initialised:
            super().__setattr__(key, value)
        else:
            """Set attributes on minion after initialisation"""
            setattr(self.minion, key, value)

In [67]:
elf = Elf()
a = MinionAdapter(elf, call = 'nall_nin')
b = MinionAdapter(Dwarf(), call = 'estver_narho')
c = MinionAdapter(Human(), call = 'ring_mig')

In [69]:
a.name = '100'

In [70]:
elf.name

'100'

In [66]:
a.call()
b.call()
c.call()

Elf says: Calling the Overlord ...
Dwarf says: Calling the Overlord ...
Human says: Calling the Overlord ...


## Observer
 - The observer pattern is a software design pattern in which an object, called the subject, maintains a list of its dependents, called observers, and notifies them automatically of any state changes, usually by calling one of their methods. (Wikipedia)
 - Assume we have problem that we have a read-only instance which has many intermediate data attributes. Users have access to modify those attributes, but it is not encouraged. How can we give a warning or raise an error when users attempt to write them?

In [72]:
class MCEngine:
    def __init__(self, num):
        self.simulation_times = num
    def simulate(self):
        print(f'Monte-Carlo simulation ({self.simulation_times} times) is going on. ')

In [74]:
engine = MCEngine(100)
engine.simulate()

Monte-Carlo simulation (100 times) is going on. 


In [75]:
# User can modify attributes from out side of this class.
engine = MCEngine(100)
engine.simulation_times = 1000
engine.simulate()

Monte-Carlo simulation (1000 times) is going on. 


In [85]:
# We can use observer to solve this:
# Firstly, we give two base classes.

class Observer:
    def __init__(self):
        self.logger = []
    def update(self, obj, *args, **kwargs):
        message = f'Object: {obj}, Args: {args}, Kwargs: {kwargs}'
        print(message)
        self.logger.append(message)


class Observable:
    def __init__(self):
        self._observers = []

    def add_observer(self, observer):
        self._observers.append(observer)

    def remove_observer(self, observer):
        self._observers.remove(observer)

    def notify_observer(self, *args, **kwargs):
        for observer in self._observers:
            observer.update(self, *args, **kwargs)

In [87]:
class ObservableAdapter(Observable):
    _initialised = False

    def __init__(self, obj):
        Observable.__init__(self)
        self.obj = obj
        self._initialised = True
        
    def __getattr__(self, attr):
        return getattr(self.obj, attr)

    def __setattr__(self, key, value):
        if not self._initialised:
            super().__setattr__(key, value)
        else:
            setattr(self.obj, key, value)
            self.notify_observer(self.obj, key, value)

In [89]:
engine = ObservableAdapter(MCEngine(100))
simulation_monitor = Observer()
engine.add_observer(simulation_monitor)

engine.simulation_times = 1000
engine.simulation_times = 10000

engine.simulate()

Object: <__main__.ObservableAdapter object at 0x000001C00D4465F8>, Args: (<__main__.MCEngine object at 0x000001C00D446630>, 'simulation_times', 1000), Kwargs: {}
Object: <__main__.ObservableAdapter object at 0x000001C00D4465F8>, Args: (<__main__.MCEngine object at 0x000001C00D446630>, 'simulation_times', 10000), Kwargs: {}
Monte-Carlo simulation (10000 times) is going on. 


In [90]:
for message in simulation_monitor.logger:
    print(message)

Object: <__main__.ObservableAdapter object at 0x000001C00D4465F8>, Args: (<__main__.MCEngine object at 0x000001C00D446630>, 'simulation_times', 1000), Kwargs: {}
Object: <__main__.ObservableAdapter object at 0x000001C00D4465F8>, Args: (<__main__.MCEngine object at 0x000001C00D446630>, 'simulation_times', 10000), Kwargs: {}


## Factory Method & Singleton
 - Assume we have a set of classes (Square, Rectangular, Circle), we want a unique global 'factory' such that I can create different instances by pass different strings to this 'factory'. To achieve this, we need this 'factory' has following bahaviour:
  1. This factory allows me to create 'Square' instance just by doing something like: factory('square', length)
  1. This factory allows me register new classes, e.g. factory.register(circle=Circle)
  1. This factory is unique (only one factory instance exists in gloabal environment) \[Singleton\]

In [93]:
class Shape:
    def area(self):
        raise NotImplementedError

In [98]:
from numpy import pi

class Square(Shape):
    def __init__(self, length):
        self.length = length
        
    @overrides(Shape)
    def area(self):
        return self.length**2
    
class Rectangular(Shape):
    def __init__(self, length, width):
        self.length = length
        self.width = width
    @overrides(Shape)
    def area(self):
        return self.length*self.width
    
class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
        
    @overrides(Shape)
    def area(self):
        return pi*self.radius**2
    
print(Square(4).area())
print(Rectangular(3,4).area())
print(Circle(4).area())

16
12
50.26548245743669


In [106]:
# We achieve (1) by hard coding a function.
def factory(name, *args, **kwargs):
    if name == 'square':
        return Square(*args, **kwargs)
    elif name == 'rectangular':
        return Rectangular(*args, **kwargs)
    elif name == 'circle':
        return Circle(*args, **kwargs)
    else:
        raise TypeError
        
print(factory('square', 4).area())
print(factory('rectangular', 3,4).area())
print(factory('circle', 4).area())


16
12
50.26548245743669


### `__new__` method
 - `__new__` method is a new feature for Python 3. It is to create a new instance of class cls. `__new__()` is a static method (special-cased so you need not declare it as such) that takes the class of which an instance was requested as its first argument.
 - If `__new__` method is not specified, it will return a instance of corresponding class, then the returned object's `__init__` method will be called.
 - You can also return other objects in `__new__()` method.(`__init__()` method can only return None)

In [110]:
class A:
    def __new__(cls):
        print('__new__ is called')
        return super().__new__(cls)
    def __init__(self):
        print('__init__ is called')
#         return 1
        
A()

__new__ is called
__init__ is called


In [111]:
class factory:
    def __new__(cls, name, *args, **kwargs):
        if name == 'square':
            return Square(*args, **kwargs)
        elif name == 'rectangular':
            return Rectangular(*args, **kwargs)
        elif name == 'circle':
            return Circle(*args, **kwargs)
        else:
            raise TypeError(f'"{name}" is not one key in this factory')
        
            

print(factory('square', 4).area())
print(factory('rectangular', 3,4).area())
print(factory('circle', 4).area())

16
12
50.26548245743669


In [135]:
# To achieve (2)
class Factory:
    def __init__(self):
        self.element = {}
        
    def RegisterInstance(self, **kwargs):
        for key, value in kwargs.items():
            self.element[key] = value
            print(f'Factory infomation: {value} is registered as "{key}"')
            
    def CreateInstance(self, name, *args, **kwargs):
        if name in self.element:
            return self.element[name](*args, **kwargs)
        else:
            raise TypeError(f'"{name}" is not one key in this factory')
            
    def RemoveKey(self, *args):
        for arg in args:
            del self.element[arg]
            print(f'Factory message: key({arg}) is removed. ')

shapeFactory = Factory()

shapeFactory.RegisterInstance(square = Square,
                         rectangular = Rectangular,
                         circle = Circle)

print(shapeFactory.CreateInstance('square', 4).area())
print(shapeFactory.CreateInstance('rectangular', 3,4).area())
print(shapeFactory.CreateInstance('circle', 4).area())

shapeFactory.RemoveKey('circle')
print(shapeFactory.CreateInstance('circle', 4).area())

Factory infomation: <class '__main__.Square'> is registered as "square"
Factory infomation: <class '__main__.Rectangular'> is registered as "rectangular"
Factory infomation: <class '__main__.Circle'> is registered as "circle"
16
12
50.26548245743669
Factory message: key(circle) is removed.? 


TypeError: 

In [137]:
# Instead, we can use class object and __new__ method as well. 
# Since we use class object, it is global and unique, thus (3) is achieved.
class factory:
    __element = {}
    def __new__(cls, name, *args, **kwargs):
        if name in factory.__element:
            return factory.__element[name](*args, **kwargs)
        else:
            raise TypeError(f'"{name}" is not one key in this factory')
            
    @classmethod
    def RegisterKey(cls, **kwargs):
        for key, value in kwargs.items():
            cls.__element[key] = value
            print(f'Factory infomation: {value} is registered as "{key}"')
    
    @classmethod
    def RemoveKey(cls, *args):
        for arg in args:
            del cls.__element[arg]
            print(f'Factory message: key({arg}) is removed. ')
      

factory.RegisterKey(square = Square,
                         rectangular = Rectangular,
                         circle = Circle)

print(factory('square', 4).area())
print(factory('rectangular', 3,4).area())
print(factory('circle', 4).area())

factory.RemoveKey('circle')
print(factory('circle', 4).area())

Factory infomation: <class '__main__.Square'> is registered as "square"
Factory infomation: <class '__main__.Rectangular'> is registered as "rectangular"
Factory infomation: <class '__main__.Circle'> is registered as "circle"
16
12
50.26548245743669
Factory message: key(circle) is removed. 


TypeError: 

In [131]:
# Another approach to achieve (3) with Singleton pattern
# Reference: https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html
# Singleton/SingletonPattern.py

class OnlyOne:
    class __OnlyOne:
        def __init__(self, arg):
            self.val = arg
        def __str__(self):
            return repr(self) + self.val
    instance = None
    def __init__(self, arg):
        if not self.instance:
            OnlyOne.instance = OnlyOne.__OnlyOne(arg)
        else:
            OnlyOne.instance.val = arg
    def __getattr__(self, name):
        return getattr(self.instance, name)
    
    def __repr__(self):
        return str(self.instance)

x = OnlyOne('sausage')
print(x)
y = OnlyOne('eggs')
print(y)
z = OnlyOne('spam')
print(z)
print(x)
print(y)

<__main__.OnlyOne.__OnlyOne object at 0x000001C00D4B5BA8>sausage
<__main__.OnlyOne.__OnlyOne object at 0x000001C00D4B5BA8>eggs
<__main__.OnlyOne.__OnlyOne object at 0x000001C00D4B5BA8>spam
<__main__.OnlyOne.__OnlyOne object at 0x000001C00D4B5BA8>spam
<__main__.OnlyOne.__OnlyOne object at 0x000001C00D4B5BA8>spam


In [143]:
class Factory:
    class __factory:
        def __init__(self):
            self.element = {}

        def RegisterInstance(self, **kwargs):
            for key, value in kwargs.items():
                self.element[key] = value
                print(f'Factory infomation: {value} is registered as "{key}"')

        def CreateInstance(self, name, *args, **kwargs):
            if name in self.element:
                return self.element[name](*args, **kwargs)
            else:
                raise TypeError(f'"{name}" is not one key in this factory')
                
        def RemoveKey(self, *args):
            for arg in args:
                del self.element[arg]
                print(f'Factory message: key({arg}) is removed. ')
                
    instance = None
    def __init__(self):
        if not Factory.instance:
            Factory.instance = Factory.__factory()
        else:
            pass
        
    def __getattr__(self, name):
        return getattr(self.instance, name)
    
    def __repr__(self):
        return str(self.instance)
    
shapeFactory = Factory()

shapeFactory.RegisterInstance(square = Square,
                         rectangular = Rectangular,
                         circle = Circle)

print(shapeFactory.CreateInstance('square', 4).area())
print(shapeFactory.CreateInstance('rectangular', 3,4).area())
print(shapeFactory.CreateInstance('circle', 4).area())

shapeFactory.RemoveKey('circle')
print(shapeFactory.CreateInstance('circle', 4).area())

Factory infomation: <class '__main__.Square'> is registered as "square"
Factory infomation: <class '__main__.Rectangular'> is registered as "rectangular"
Factory infomation: <class '__main__.Circle'> is registered as "circle"
16
12
50.26548245743669
Factory message: key(circle) is removed. 


TypeError: "circle" is not one key in this factory